# CS777 HW2 Using Spark DataFrame and Spark SQL
 - For each part, show both the DataFrame and SQL solutions

In [1]:
from pyspark import SparkContext

from pyspark.sql import SparkSession

from pyspark.sql.window import Window

from pyspark.sql.functions import col, expr, when,\
desc, first, last, row_number, rank, substring

from pyspark.sql.functions import max as pmax, min as pmin

In [2]:
import wget

In [3]:
fileName = 'https://people.bu.edu/kalathur/datasets/stocks_2013_2021.txt'
wget.download(fileName)

'stocks_2013_2021 (1).txt'

In [5]:
spark = SparkSession.Builder().appName(
    "RDD_DataFrames01").getOrCreate()
spark

In [6]:
stocksDF = spark.read.csv(
    "stocks_2013_2021.txt",
    header=False, inferSchema=True,
).toDF('Stock','Date','Open','High',
       'Low','Close','Volume'
      )

stocksDF.printSchema()

stocksDF.createOrReplaceTempView('stocks_table')

root
 |-- Stock: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



**File format**:

Stock,Date,Open,High,Low,Close,Volume

## Part1 (10 points)
 - a) Show the first two entries
 - b) How many total entries are there in the dataset?
 - c) How many entries are there in the dataset for each Stock?
 
 
Sample output for a)
 
 ```{}
 +-----+----------+-----+-----+-----+-----+---------+
|Stock|      Date| Open| High|  Low|Close|   Volume|
+-----+----------+-----+-----+-----+-----+---------+
| AAPL|2013-01-02|19.78|19.82|19.34|16.96|560518000|
| AAPL|2013-01-03|19.57|19.63|19.32|16.75|352965200|
+-----+----------+-----+-----+-----+-----+---------+

```
 
Sample output for c)

```{}
+-----+-----+
|Stock|count|
+-----+-----+
| AAPL| 2266|
| GOOG| 2266|
| INTC| 2266|
| MSFT| 2266|
| NFLX| 2266|
| NVDA| 2266|
+-----+-----+

```
 

### DataFrame Solutions

In [7]:
stocksDF.show(2)

+-----+----------+-----+-----+-----+-----+---------+
|Stock|      Date| Open| High|  Low|Close|   Volume|
+-----+----------+-----+-----+-----+-----+---------+
| AAPL|2013-01-02|19.78|19.82|19.34|16.96|560518000|
| AAPL|2013-01-03|19.57|19.63|19.32|16.75|352965200|
+-----+----------+-----+-----+-----+-----+---------+
only showing top 2 rows



In [10]:
stocksDF.count()

13596

In [12]:
stocksDF.groupBy('Stock').count().show()

+-----+-----+
|Stock|count|
+-----+-----+
| AAPL| 2266|
| INTC| 2266|
| GOOG| 2266|
| NFLX| 2266|
| MSFT| 2266|
| NVDA| 2266|
+-----+-----+



### SQL Solutions

In [135]:
spark.sql("""
SELECT * FROM stocks_table

""").show(2)

+-----+----------+-----+-----+-----+-----+---------+
|Stock|      Date| Open| High|  Low|Close|   Volume|
+-----+----------+-----+-----+-----+-----+---------+
| AAPL|2013-01-02|19.78|19.82|19.34|16.96|560518000|
| AAPL|2013-01-03|19.57|19.63|19.32|16.75|352965200|
+-----+----------+-----+-----+-----+-----+---------+
only showing top 2 rows



In [138]:
spark.sql("""
SELECT count(*) AS freq
    FROM stocks_table

""").show()

+-----+
| freq|
+-----+
|13596|
+-----+



In [140]:
   spark.sql("""
SELECT stock, count(*) AS freq
    FROM stocks_table
    GROUP BY Stock

""").show()                            

+-----+----+
|stock|freq|
+-----+----+
| AAPL|2266|
| INTC|2266|
| GOOG|2266|
| NFLX|2266|
| MSFT|2266|
| NVDA|2266|
+-----+----+



## Part2 (10 points)
 - a) Show only the Stock and Volume columns.
 - b) What is the total volume for each stock symbol in the dataset
 - c) Show the results in the descending order of trading volume
 
Sample output for a)

```{}
+-----+---------+
|Stock|   Volume|
+-----+---------+
| AAPL|560518000|
| AAPL|352965200|
+-----+---------+
only showing top 2 rows
```

Sample output for b)

```{}
+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| INTC| 65334389200|
| GOOG|  4626409870|
| NFLX| 28981935000|
| MSFT| 73349882000|
| NVDA| 98721855200|
+-----+------------+
```

Sample output for c)

```{}
+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| NVDA| 98721855200|
| MSFT| 73349882000|
| INTC| 65334389200|
| NFLX| 28981935000|
| GOOG|  4626409870|
+-----+------------+
```



### DataFrame Solutions

In [14]:
stocksDF.select(['Stock','Volume']).show(2)

+-----+---------+
|Stock|   Volume|
+-----+---------+
| AAPL|560518000|
| AAPL|352965200|
+-----+---------+
only showing top 2 rows



In [24]:
stocksDF.groupBy('Stock').sum('Volume').select('Stock',col('sum(Volume)').alias('Sum')).show()

+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| INTC| 65334389200|
| GOOG|  4626409870|
| NFLX| 28981935000|
| MSFT| 73349882000|
| NVDA| 98721855200|
+-----+------------+



In [23]:
stocksDF.groupBy('Stock').sum('Volume').select('Stock',col('sum(Volume)').alias('Sum')).sort('Sum',ascending=False).show()

+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| NVDA| 98721855200|
| MSFT| 73349882000|
| INTC| 65334389200|
| NFLX| 28981935000|
| GOOG|  4626409870|
+-----+------------+



### SQL Solutions

In [141]:
spark.sql("""
SELECT Stock,Volume FROM stocks_table

""").show(2)

+-----+---------+
|Stock|   Volume|
+-----+---------+
| AAPL|560518000|
| AAPL|352965200|
+-----+---------+
only showing top 2 rows



In [144]:
spark.sql("""
SELECT Stock,sum(Volume) as Sum 
    FROM stocks_table
    GROUP BY Stock

""").show()

+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| INTC| 65334389200|
| GOOG|  4626409870|
| NFLX| 28981935000|
| MSFT| 73349882000|
| NVDA| 98721855200|
+-----+------------+



In [146]:
spark.sql("""
SELECT Stock,sum(Volume) as Sum 
    FROM stocks_table
    GROUP BY Stock
    ORDER BY Sum DESC

""").show()

+-----+------------+
|Stock|         Sum|
+-----+------------+
| AAPL|409337906700|
| NVDA| 98721855200|
| MSFT| 73349882000|
| INTC| 65334389200|
| NFLX| 28981935000|
| GOOG|  4626409870|
+-----+------------+



## Part 3 (10 points)

 - a) Show only the Stock, Open, and Close columns.
 - b) For each stock, how many days over the entire dataset did that stock's closing price was above the opening price? Show the results in descreasing order as shown below. First create Higher column (1/0), i.e., 1 if the condition is true, 0 if false. Use this column for computing the results.
 
 Sample output for a)
 
 ```{}
+-----+-----+-----+
|Stock| Open|Close|
+-----+-----+-----+
| AAPL|19.78|16.96|
| AAPL|19.57|16.75|
+-----+-----+-----+
only showing top 2 rows
```

Sample output for b)

```{}
+-----+------+
|Stock|Higher|
+-----+------+
| GOOG|  1166|
| NFLX|  1125|
| NVDA|   525|
| AAPL|   202|
| MSFT|   153|
| INTC|    55|
+-----+------+
```

### DataFrame Solutions

In [25]:
stocksDF.select('Stock','Open','Close').show(2)

+-----+-----+-----+
|Stock| Open|Close|
+-----+-----+-----+
| AAPL|19.78|16.96|
| AAPL|19.57|16.75|
+-----+-----+-----+
only showing top 2 rows



In [34]:
stocksDF.select(stocksDF.Stock,
                when(stocksDF.Close > stocksDF.Open, 1).
                otherwise(0).alias('Higher')).groupBy(stocksDF.Stock).sum('Higher').withColumnRenamed('sum(Higher)','Higher').sort('Higher',ascending=False).show()



+-----+------+
|Stock|Higher|
+-----+------+
| GOOG|  1166|
| NFLX|  1125|
| NVDA|   525|
| AAPL|   202|
| MSFT|   153|
| INTC|    55|
+-----+------+



### SQL Solutions

In [155]:
spark.sql("""
SELECT Stock, Open, Close
    FROM stocks_table

""").show(2)

+-----+-----+-----+
|Stock| Open|Close|
+-----+-----+-----+
| AAPL|19.78|16.96|
| AAPL|19.57|16.75|
+-----+-----+-----+
only showing top 2 rows



In [157]:
spark.sql("""
SELECT Stock, SUM(IF(Close>Open,1,0)) as freq
    FROM stocks_table
    GROUP BY  Stock
    ORDER BY freq DESC

""").show()

+-----+----+
|Stock|freq|
+-----+----+
| GOOG|1166|
| NFLX|1125|
| NVDA| 525|
| AAPL| 202|
| MSFT| 153|
| INTC|  55|
+-----+----+



## Part4 (10 points)
 - a) Show only the Stock, High, and Low columns.
 - b) For each stock, what is the largest High and the smallest Low price?
 
Sample output for a)
 
 ```{}
+-----+-----+-----+
|Stock| High|  Low|
+-----+-----+-----+
| AAPL|19.82|19.34|
| AAPL|19.63|19.32|
+-----+-----+-----+
only showing top 2 rows
 ```
 
Sample output for b)
 
 ```{}
+-----+---------+--------+
|Stock|max(High)|min(Low)|
+-----+---------+--------+
| AAPL|   182.13|   13.75|
| GOOG|   3037.0|  346.46|
| INTC|    69.29|    20.1|
| MSFT|   349.67|   26.28|
| NFLX|   700.99|   12.96|
| NVDA|   346.47|    2.98|
+-----+---------+--------+
 ```
 

### DataFrame Solutions

In [35]:
stocksDF.select('Stock','High','Low').show(2)

+-----+-----+-----+
|Stock| High|  Low|
+-----+-----+-----+
| AAPL|19.82|19.34|
| AAPL|19.63|19.32|
+-----+-----+-----+
only showing top 2 rows



In [39]:
stocksDF.groupby('Stock').agg(
    pmax('High'),
    pmin('Low')).show()

+-----+---------+--------+
|Stock|max(High)|min(Low)|
+-----+---------+--------+
| AAPL|   182.13|   13.75|
| INTC|    69.29|    20.1|
| GOOG|   3037.0|  346.46|
| NFLX|   700.99|   12.96|
| MSFT|   349.67|   26.28|
| NVDA|   346.47|    2.98|
+-----+---------+--------+



### SQL Solutions

In [158]:
spark.sql("""
SELECT Stock, High, Low
    FROM stocks_table

""").show(2)

+-----+-----+-----+
|Stock| High|  Low|
+-----+-----+-----+
| AAPL|19.82|19.34|
| AAPL|19.63|19.32|
+-----+-----+-----+
only showing top 2 rows



In [160]:
spark.sql("""
SELECT Stock, max(High), min(Low)
    FROM stocks_table
    GROUP BY Stock

""").show()

+-----+---------+--------+
|Stock|max(High)|min(Low)|
+-----+---------+--------+
| AAPL|   182.13|   13.75|
| INTC|    69.29|    20.1|
| GOOG|   3037.0|  346.46|
| NFLX|   700.99|   12.96|
| MSFT|   349.67|   26.28|
| NVDA|   346.47|    2.98|
+-----+---------+--------+



## Part5 (10 points)

 - a) Show the original data as shown below. Create HighDate and LowDate columns for each row in the data using the Date column.
 - b) For each stock, what is the largest High and the smallest Low price along with that respective date? Use the following approach for the solution: Using the Window partition and order operations with row_number, compute separately the HighDate for the largest High price and the LowDate for the smallest Low price and join the two results.
 
Sample output for a)

```{}
+-----+-----+----------+-----+----------+
|Stock| High|  HighDate|  Low|   LowDate|
+-----+-----+----------+-----+----------+
| AAPL|19.82|2013-01-02|19.34|2013-01-02|
| AAPL|19.63|2013-01-03|19.32|2013-01-03|
| AAPL|19.24|2013-01-04|18.78|2013-01-04|
| AAPL| 18.9|2013-01-07| 18.4|2013-01-07|
| AAPL| 19.0|2013-01-08|18.62|2013-01-08|
+-----+-----+----------+-----+----------+
only showing top 5 rows
```

Sample output for b)

```{}
+-----+------+----------+------+----------+
|Stock|  High|  HighDate|   Low|   LowDate|
+-----+------+----------+------+----------+
| AAPL|182.13|2021-12-13| 13.75|2013-04-19|
| GOOG|3037.0|2021-11-19|346.46|2013-01-22|
| INTC| 69.29|2020-01-24|  20.1|2013-02-22|
| MSFT|349.67|2021-11-22| 26.28|2013-01-11|
| NFLX|700.99|2021-11-17| 12.96|2013-01-02|
| NVDA|346.47|2021-11-22|  2.98|2013-01-15|
+-----+------+----------+------+----------+
```


### DataFrame Solutions

In [50]:
HighLow = stocksDF.withColumn('HighDate',col('Date'))\
        .withColumn('LowDate',col('Date'))\
        .select('Stock','High','HighDate','Low','LowDate')

HighLow.show(5)




+-----+-----+----------+-----+----------+
|Stock| High|  HighDate|  Low|   LowDate|
+-----+-----+----------+-----+----------+
| AAPL|19.82|2013-01-02|19.34|2013-01-02|
| AAPL|19.63|2013-01-03|19.32|2013-01-03|
| AAPL|19.24|2013-01-04|18.78|2013-01-04|
| AAPL| 18.9|2013-01-07| 18.4|2013-01-07|
| AAPL| 19.0|2013-01-08|18.62|2013-01-08|
+-----+-----+----------+-----+----------+
only showing top 5 rows



In [55]:
windowSpec = Window.partitionBy("Stock").orderBy(desc('High'))
windowSpec2 = Window.partitionBy("Stock").orderBy('High')

High = HighLow.withColumn(
    "row", row_number().over(windowSpec)
).filter("row == 1").select('Stock','High','HighDate')

Low = HighLow.withColumn(
    "row", row_number().over(windowSpec2)
).filter("row == 1").select('Stock','Low','LowDate')

High.join(Low, on = 'Stock').show()

+-----+------+----------+------+----------+
|Stock|  High|  HighDate|   Low|   LowDate|
+-----+------+----------+------+----------+
| AAPL|182.13|2021-12-13| 13.75|2013-04-19|
| GOOG|3037.0|2021-11-19|346.46|2013-01-22|
| INTC| 69.29|2020-01-24|  20.1|2013-02-22|
| MSFT|349.67|2021-11-22| 26.56|2013-01-09|
| NFLX|700.99|2021-11-17| 12.96|2013-01-02|
| NVDA|346.47|2021-11-22|  2.98|2013-01-15|
+-----+------+----------+------+----------+



### SQL Solutions

In [212]:
df_1 = spark.sql("""
SELECT Stock, High, CAST(Date AS DATE) as HighDate,Low, CAST(Date as DATE) as LowDate
    FROM stocks_table

""")

df_1.show(5)

+-----+-----+----------+-----+----------+
|Stock| High|  HighDate|  Low|   LowDate|
+-----+-----+----------+-----+----------+
| AAPL|19.82|2013-01-02|19.34|2013-01-02|
| AAPL|19.63|2013-01-03|19.32|2013-01-03|
| AAPL|19.24|2013-01-04|18.78|2013-01-04|
| AAPL| 18.9|2013-01-07| 18.4|2013-01-07|
| AAPL| 19.0|2013-01-08|18.62|2013-01-08|
+-----+-----+----------+-----+----------+
only showing top 5 rows



In [219]:
df_1.createOrReplaceTempView('table1')


High = spark.sql("""
SELECT Stock, High, HighDate 
FROM
  (SELECT *,
    row_number()       OVER(PARTITION BY Stock ORDER BY High DESC) AS row
    FROM table1) 
    WHERE row == 1
""")

Low = spark.sql("""
SELECT Stock, Low, LowDate 
FROM
  (SELECT *,
    row_number()       OVER(PARTITION BY Stock ORDER BY Low ASC) AS row
    FROM table1) 
    WHERE row == 1
""")

High.createOrReplaceTempView('tableH')
Low.createOrReplaceTempView('tableL')

spark.sql("""
    SELECT tableH.Stock, High, HighDate, Low, LowDate FROM 
        tableH INNER JOIN tableL
            ON tableH.Stock = tableL.Stock
""").show()

+-----+------+----------+------+----------+
|Stock|  High|  HighDate|   Low|   LowDate|
+-----+------+----------+------+----------+
| AAPL|182.13|2021-12-13| 13.75|2013-04-19|
| GOOG|3037.0|2021-11-19|346.46|2013-01-22|
| INTC| 69.29|2020-01-24|  20.1|2013-02-22|
| MSFT|349.67|2021-11-22| 26.28|2013-01-11|
| NFLX|700.99|2021-11-17| 12.96|2013-01-02|
| NVDA|346.47|2021-11-22|  2.98|2013-01-15|
+-----+------+----------+------+----------+



## Part 6 (10 points)
 The following will use (Stock, Year) as the key.
 
 - a) Show the Stock, Year, and Volume data as shown below.
 - b) What is the total volume for each stock symbol per each year in the dataset
 
- Sample output for a)

```{}
+-----+----+---------+
|Stock|Year|   Volume|
+-----+----+---------+
| AAPL|2013|560518000|
| AAPL|2013|352965200|
| AAPL|2013|594333600|
| AAPL|2013|484156400|
| AAPL|2013|458707200|
+-----+----+---------+
only showing top 5 rows
```

- Sample output for b)

```{}
+-----+----+------------+
|Stock|Year| sum(Volume)|
+-----+----+------------+
| AAPL|2013|102421569600|
| AAPL|2014| 63657952400|
| AAPL|2015| 52264199600|
| AAPL|2016| 38729911200|
| AAPL|2017| 27243106000|
| AAPL|2018| 34156144800|
| AAPL|2019| 28254942800|
| AAPL|2020| 39863855600|
| AAPL|2021| 22746224700|
| GOOG|2013|  1055993341|
| GOOG|2014|   626749011|
| GOOG|2015|   521501318|
| GOOG|2016|   461731000|
| GOOG|2017|   370605100|
| GOOG|2018|   437233300|
| GOOG|2019|   356294900|
| GOOG|2020|   478595100|
| GOOG|2021|   317706800|
| INTC|2013|  9856167600|
| INTC|2014|  7965304500|
+-----+----+------------+
only showing top 20 rows
```


### DataFrame Solutions

In [64]:
stocksDF.selectExpr('Stock','year(Date) as Year','Volume').show(5)

+-----+----+---------+
|Stock|Year|   Volume|
+-----+----+---------+
| AAPL|2013|560518000|
| AAPL|2013|352965200|
| AAPL|2013|594333600|
| AAPL|2013|484156400|
| AAPL|2013|458707200|
+-----+----+---------+
only showing top 5 rows



In [72]:
stocksDF.selectExpr('Stock','year(Date) as Year','Volume').groupBy('Stock','Year').sum('Volume').sort('Stock','Year').show(20)


+-----+----+------------+
|Stock|Year| sum(Volume)|
+-----+----+------------+
| AAPL|2013|102421569600|
| AAPL|2014| 63657952400|
| AAPL|2015| 52264199600|
| AAPL|2016| 38729911200|
| AAPL|2017| 27243106000|
| AAPL|2018| 34156144800|
| AAPL|2019| 28254942800|
| AAPL|2020| 39863855600|
| AAPL|2021| 22746224700|
| GOOG|2013|  1055993341|
| GOOG|2014|   626749011|
| GOOG|2015|   521501318|
| GOOG|2016|   461731000|
| GOOG|2017|   370605100|
| GOOG|2018|   437233300|
| GOOG|2019|   356294900|
| GOOG|2020|   478595100|
| GOOG|2021|   317706800|
| INTC|2013|  9856167600|
| INTC|2014|  7965304500|
+-----+----+------------+
only showing top 20 rows



### SQL Solutions

In [169]:
spark.sql("""
SELECT Stock, year(Date) as Year, Volume
    FROM stocks_table

""").show(5)

+-----+----+---------+
|Stock|Year|   Volume|
+-----+----+---------+
| AAPL|2013|560518000|
| AAPL|2013|352965200|
| AAPL|2013|594333600|
| AAPL|2013|484156400|
| AAPL|2013|458707200|
+-----+----+---------+
only showing top 5 rows



In [179]:
spark.sql("""
SELECT Stock, year(Date) as Year, sum(Volume)
    FROM stocks_table
    GROUP BY Stock, Year 
    ORDER BY Stock, Year ASC

""").show()

+-----+----+------------+
|Stock|Year| sum(Volume)|
+-----+----+------------+
| AAPL|2013|102421569600|
| AAPL|2014| 63657952400|
| AAPL|2015| 52264199600|
| AAPL|2016| 38729911200|
| AAPL|2017| 27243106000|
| AAPL|2018| 34156144800|
| AAPL|2019| 28254942800|
| AAPL|2020| 39863855600|
| AAPL|2021| 22746224700|
| GOOG|2013|  1055993341|
| GOOG|2014|   626749011|
| GOOG|2015|   521501318|
| GOOG|2016|   461731000|
| GOOG|2017|   370605100|
| GOOG|2018|   437233300|
| GOOG|2019|   356294900|
| GOOG|2020|   478595100|
| GOOG|2021|   317706800|
| INTC|2013|  9856167600|
| INTC|2014|  7965304500|
+-----+----+------------+
only showing top 20 rows



## Part7 (20 points)

- a) Show the Stock, Year, High, and Low values as shown below.
- b) For each stock per each year, what is the largest High and the smallest Low price
- c) For the above problem, show the dates for the largest High and the smallest Low for each stock per each year. Use the same approach as in 5b.


- Sample output for a)

```{}
+-----+----+-----+-----+
|Stock|Year| High|  Low|
+-----+----+-----+-----+
| AAPL|2013|19.82|19.34|
| AAPL|2013|19.63|19.32|
+-----+----+-----+-----+
only showing top 2 rows
```

- Sample output for b)

```{}
+-----+----+---------+--------+
|Stock|Year|max(High)|min(Low)|
+-----+----+---------+--------+
| AAPL|2013|    20.54|   13.75|
| AAPL|2014|    29.94|   17.63|
| AAPL|2015|    33.63|    23.0|
| AAPL|2016|    29.67|   22.37|
| AAPL|2017|     44.3|   28.69|
| AAPL|2018|    58.37|   36.65|
| AAPL|2019|    73.49|    35.5|
| AAPL|2020|   138.79|   53.15|
| AAPL|2021|   182.13|  116.21|
| GOOG|2013|   558.41|  346.46|
| GOOG|2014|   612.15|  487.66|
| GOOG|2015|   779.98|  486.23|
| GOOG|2016|   816.68|  663.06|
| GOOG|2017|  1078.49|   775.8|
| GOOG|2018|  1273.89|  970.11|
| GOOG|2019|   1365.0| 1014.07|
| GOOG|2020|   1847.2| 1013.54|
| GOOG|2021|   3037.0|  1699.0|
| INTC|2013|    26.04|    20.1|
| INTC|2014|     37.9|    23.5|
+-----+----+---------+--------+
only showing top 20 rows

```

- Sample output for c)

```{}
+-----+----+-------+----------+-------+----------+
|Stock|Year|   High|  HighDate|    Low|   LowDate|
+-----+----+-------+----------+-------+----------+
| AAPL|2013|  20.54|2013-12-05|  13.75|2013-04-19|
| AAPL|2014|  29.94|2014-11-25|  17.63|2014-01-31|
| AAPL|2015|  33.63|2015-04-28|   23.0|2015-08-24|
| AAPL|2016|  29.67|2016-10-11|  22.37|2016-05-12|
| AAPL|2017|   44.3|2017-12-18|  28.69|2017-01-03|
| AAPL|2018|  58.37|2018-10-03|  36.65|2018-12-24|
| AAPL|2019|  73.49|2019-12-27|   35.5|2019-01-03|
| AAPL|2020| 138.79|2020-12-29|  53.15|2020-03-23|
| AAPL|2021| 182.13|2021-12-13| 116.21|2021-03-08|
| GOOG|2013| 558.41|2013-12-31| 346.46|2013-01-22|
| GOOG|2014| 612.15|2014-02-26| 487.66|2014-12-16|
| GOOG|2015| 779.98|2015-12-29| 486.23|2015-01-12|
| GOOG|2016| 816.68|2016-10-25| 663.06|2016-02-08|
| GOOG|2017|1078.49|2017-12-18|  775.8|2017-01-03|
| GOOG|2018|1273.89|2018-07-27| 970.11|2018-12-24|
| GOOG|2019| 1365.0|2019-12-17|1014.07|2019-01-03|
| GOOG|2020| 1847.2|2020-12-03|1013.54|2020-03-23|
| GOOG|2021| 3037.0|2021-11-19| 1699.0|2021-01-06|
| INTC|2013|  26.04|2013-12-31|   20.1|2013-02-22|
| INTC|2014|   37.9|2014-12-05|   23.5|2014-02-05|
+-----+----+-------+----------+-------+----------+
only showing top 20 rows
```


### DataFrame Solutions

In [74]:
stocksDF.selectExpr('Stock','year(Date) as Year','High','Low').show(2)

+-----+----+-----+-----+
|Stock|Year| High|  Low|
+-----+----+-----+-----+
| AAPL|2013|19.82|19.34|
| AAPL|2013|19.63|19.32|
+-----+----+-----+-----+
only showing top 2 rows



In [78]:
stocksDF.selectExpr('Stock','year(Date) as Year','High','Low').groupBy('Stock','Year').agg(
    pmax('High'), pmin('Low')).sort('Stock','Year').show(20)

+-----+----+---------+--------+
|Stock|Year|max(High)|min(Low)|
+-----+----+---------+--------+
| AAPL|2013|    20.54|   13.75|
| AAPL|2014|    29.94|   17.63|
| AAPL|2015|    33.63|    23.0|
| AAPL|2016|    29.67|   22.37|
| AAPL|2017|     44.3|   28.69|
| AAPL|2018|    58.37|   36.65|
| AAPL|2019|    73.49|    35.5|
| AAPL|2020|   138.79|   53.15|
| AAPL|2021|   182.13|  116.21|
| GOOG|2013|   558.41|  346.46|
| GOOG|2014|   612.15|  487.66|
| GOOG|2015|   779.98|  486.23|
| GOOG|2016|   816.68|  663.06|
| GOOG|2017|  1078.49|   775.8|
| GOOG|2018|  1273.89|  970.11|
| GOOG|2019|   1365.0| 1014.07|
| GOOG|2020|   1847.2| 1013.54|
| GOOG|2021|   3037.0|  1699.0|
| INTC|2013|    26.04|    20.1|
| INTC|2014|     37.9|    23.5|
+-----+----+---------+--------+
only showing top 20 rows



In [81]:
windowSpec3 = Window.partitionBy("Stock",'Year').orderBy(desc('High'))
windowSpec4 = Window.partitionBy("Stock",'Year').orderBy('High')

HighLow = stocksDF.withColumn('HighDate',col('Date'))\
        .withColumn('LowDate',col('Date'))\
        .selectExpr('Stock','year(Date) as Year','High','HighDate','Low','LowDate')

High = HighLow.withColumn(
    "row", row_number().over(windowSpec3)
).filter("row == 1").select('Stock','Year','High','HighDate')

Low = HighLow.withColumn(
    "row", row_number().over(windowSpec4)
).filter("row == 1").select('Stock','Year','Low','LowDate')

High.join(Low, on=['Stock','Year']).show()



+-----+----+-------+----------+-------+----------+
|Stock|Year|   High|  HighDate|    Low|   LowDate|
+-----+----+-------+----------+-------+----------+
| AAPL|2013|  20.54|2013-12-05|  13.75|2013-04-19|
| AAPL|2014|  29.94|2014-11-25|  17.63|2014-01-31|
| AAPL|2015|  33.63|2015-04-28|   26.2|2015-12-31|
| AAPL|2016|  29.67|2016-10-11|   22.5|2016-05-13|
| AAPL|2017|   44.3|2017-12-18|  28.69|2017-01-03|
| AAPL|2018|  58.37|2018-10-03|  36.65|2018-12-24|
| AAPL|2019|  73.49|2019-12-27|   35.5|2019-01-03|
| AAPL|2020| 138.79|2020-12-29|  53.15|2020-03-23|
| AAPL|2021| 182.13|2021-12-13| 118.86|2021-03-30|
| GOOG|2013| 558.41|2013-12-31| 346.46|2013-01-22|
| GOOG|2014| 612.15|2014-02-26| 495.45|2014-12-17|
| GOOG|2015| 779.98|2015-12-29| 486.23|2015-01-12|
| GOOG|2016| 816.68|2016-10-25| 663.28|2016-06-27|
| GOOG|2017|1078.49|2017-12-18|  775.8|2017-01-03|
| GOOG|2018|1273.89|2018-07-27| 970.11|2018-12-24|
| GOOG|2019| 1365.0|2019-12-17| 1033.7|2019-06-06|
| GOOG|2020| 1847.2|2020-12-03|

### SQL Solutions

In [175]:
spark.sql("""
SELECT Stock, year(Date) as Year, High, Low
    FROM stocks_table

""").show(2)

+-----+----+-----+-----+
|Stock|Year| High|  Low|
+-----+----+-----+-----+
| AAPL|2013|19.82|19.34|
| AAPL|2013|19.63|19.32|
+-----+----+-----+-----+
only showing top 2 rows



In [178]:
spark.sql("""
SELECT Stock, year(Date) as Year, max(High), min(Low)
    FROM stocks_table
    GROUP BY Stock, Year
    ORDER BY Stock, Year ASC

""").show()

+-----+----+---------+--------+
|Stock|Year|max(High)|min(Low)|
+-----+----+---------+--------+
| AAPL|2013|    20.54|   13.75|
| AAPL|2014|    29.94|   17.63|
| AAPL|2015|    33.63|    23.0|
| AAPL|2016|    29.67|   22.37|
| AAPL|2017|     44.3|   28.69|
| AAPL|2018|    58.37|   36.65|
| AAPL|2019|    73.49|    35.5|
| AAPL|2020|   138.79|   53.15|
| AAPL|2021|   182.13|  116.21|
| GOOG|2013|   558.41|  346.46|
| GOOG|2014|   612.15|  487.66|
| GOOG|2015|   779.98|  486.23|
| GOOG|2016|   816.68|  663.06|
| GOOG|2017|  1078.49|   775.8|
| GOOG|2018|  1273.89|  970.11|
| GOOG|2019|   1365.0| 1014.07|
| GOOG|2020|   1847.2| 1013.54|
| GOOG|2021|   3037.0|  1699.0|
| INTC|2013|    26.04|    20.1|
| INTC|2014|     37.9|    23.5|
+-----+----+---------+--------+
only showing top 20 rows



In [241]:

High = spark.sql("""
SELECT Stock, Year, High, HighDate
FROM
  (SELECT Stock, year(Date) as Year, High ,Date as HighDate,
    row_number()       OVER(PARTITION BY Stock, year(Date) ORDER BY High DESC) AS row
    FROM stocks_table) 
    WHERE row == 1
""")

Low = spark.sql("""
SELECT Stock, Year, Low, LowDate
FROM
  (SELECT Stock, year(Date) as Year, Low, Date as LowDate,
    row_number()       OVER(PARTITION BY Stock, year(Date) ORDER BY Low ASC) AS row
    FROM stocks_table) 
    WHERE row == 1
""")

High.createOrReplaceTempView('tableH')
Low.createOrReplaceTempView('tableL')

spark.sql("""
    SELECT tableH.Stock, tableH.Year, High, HighDate, Low, LowDate FROM 
        tableH INNER JOIN tableL
            ON (tableH.Stock = tableL.Stock AND tableH.Year = tableL.Year)
""").show()

+-----+----+-------+----------+-------+----------+
|Stock|Year|   High|  HighDate|    Low|   LowDate|
+-----+----+-------+----------+-------+----------+
| AAPL|2013|  20.54|2013-12-05|  13.75|2013-04-19|
| AAPL|2014|  29.94|2014-11-25|  17.63|2014-01-31|
| AAPL|2015|  33.63|2015-04-28|   23.0|2015-08-24|
| AAPL|2016|  29.67|2016-10-11|  22.37|2016-05-12|
| AAPL|2017|   44.3|2017-12-18|  28.69|2017-01-03|
| AAPL|2018|  58.37|2018-10-03|  36.65|2018-12-24|
| AAPL|2019|  73.49|2019-12-27|   35.5|2019-01-03|
| AAPL|2020| 138.79|2020-12-29|  53.15|2020-03-23|
| AAPL|2021| 182.13|2021-12-13| 116.21|2021-03-08|
| GOOG|2013| 558.41|2013-12-31| 346.46|2013-01-22|
| GOOG|2014| 612.15|2014-02-26| 487.66|2014-12-16|
| GOOG|2015| 779.98|2015-12-29| 486.23|2015-01-12|
| GOOG|2016| 816.68|2016-10-25| 663.06|2016-02-08|
| GOOG|2017|1078.49|2017-12-18|  775.8|2017-01-03|
| GOOG|2018|1273.89|2018-07-27| 970.11|2018-12-24|
| GOOG|2019| 1365.0|2019-12-17|1014.07|2019-01-03|
| GOOG|2020| 1847.2|2020-12-03|

## Part 8 (20 points)

 - a) Filter the dataset for Google with the net percent change  of Close and Open values as shown below.
 - b) Filter the dataset for Netflix with the net percent change  of Close and Open values as shown below.
 - c) Join the above two as shown below.
 - d) Using the result from c), first filter the Google values if its net percent change is higher, and then filter the Netflix values if its net percent change is higher, and form the UNION of the two. Show the results as shown below.
 - e) using the result from d), select the Year, Stock, and net percent change as shown below.
 - f) Show the sum of the net_percent_change values for each Stock per each year.
 
 

- Sample solution for a)

```{}
+----------+------+------+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|
+----------+------+------+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|
|2013-01-04|  GOOG|363.31|367.61|      1.18|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|
+----------+------+------+------+----------+
only showing top 5 rows
```

- Sample solution for b)

```{}
+----------+------+-----+------+----------+
|      Date|Stock2|Open2|Close2|NetChange2|
+----------+------+-----+------+----------+
|2013-01-02|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  NFLX|14.29| 13.88|     -2.87|
+----------+------+-----+------+----------+
only showing top 5 rows
```

- Sample solution for c)

```{}
+----------+------+------+------+----------+------+-----+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|Stock2|Open2|Close2|NetChange2|
+----------+------+------+------+----------+------+-----+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  GOOG|363.31|367.61|      1.18|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|  NFLX|14.29| 13.88|     -2.87|
+----------+------+------+------+----------+------+-----+------+----------+
only showing top 5 rows
```

- Sample solution for d)

```{}
+----------+-----+------+------+---------+
|      Date|Stock|  Open| Close|NetChange|
+----------+-----+------+------+---------+
|2013-01-02| GOOG|358.37|360.27|     0.53|
|2013-01-03| NFLX| 13.14|  13.8|     5.02|
|2013-01-04| GOOG|363.31|367.61|     1.18|
|2013-01-07| NFLX| 13.77| 14.17|      2.9|
|2013-01-08| GOOG| 366.4|365.28|    -0.31|
|2013-01-09| GOOG|364.77|367.68|      0.8|
|2013-01-10| NFLX|  13.8|  14.0|     1.45|
|2013-01-11| NFLX| 14.03| 14.47|     3.14|
|2013-01-14| NFLX| 14.43| 14.78|     2.43|
|2013-01-15| GOOG|358.32|361.11|     0.78|
|2013-01-16| GOOG|359.85|356.26|     -1.0|
|2013-01-17| NFLX| 14.01| 13.96|    -0.36|
|2013-01-18| GOOG|353.85|350.94|    -0.82|
|2013-01-22| GOOG|351.01|350.12|    -0.25|
|2013-01-23| NFLX| 13.88| 14.75|     6.27|
|2013-01-24| NFLX| 20.57| 20.98|     1.99|
|2013-01-25| NFLX| 20.81| 24.22|    16.39|
|2013-01-28| GOOG|374.48|373.96|    -0.14|
|2013-01-29| NFLX| 22.59| 24.16|     6.95|
|2013-01-30| GOOG|375.46|375.51|     0.01|
+----------+-----+------+------+---------+
```

- Sample solution for e)

```{}
+----+-----+---------+
|Year|Stock|NetChange|
+----+-----+---------+
|2013| GOOG|     0.53|
|2013| NFLX|     5.02|
|2013| GOOG|     1.18|
|2013| NFLX|      2.9|
|2013| GOOG|    -0.31|
|2013| GOOG|      0.8|
|2013| NFLX|     1.45|
|2013| NFLX|     3.14|
|2013| NFLX|     2.43|
|2013| GOOG|     0.78|
|2013| GOOG|     -1.0|
|2013| NFLX|    -0.36|
|2013| GOOG|    -0.82|
|2013| GOOG|    -0.25|
|2013| NFLX|     6.27|
|2013| NFLX|     1.99|
|2013| NFLX|    16.39|
|2013| GOOG|    -0.14|
|2013| NFLX|     6.95|
|2013| GOOG|     0.01|
+----+-----+---------+
only showing top 20 rows

```

- Sample solution for f)

```{}
+----+-----+--------------+
|Year|Stock|TotalNetChange|
+----+-----+--------------+
|2013| GOOG|         26.02|
|2013| NFLX|        241.05|
|2014| GOOG|         -4.03|
|2014| NFLX|        141.55|
|2015| GOOG|         30.11|
|2015| NFLX|        185.94|
|2016| GOOG|          0.72|
|2016| NFLX|         168.4|
|2017| GOOG|         17.07|
|2017| NFLX|        109.73|
|2018| GOOG|          -8.9|
|2018| NFLX|        190.29|
|2019| GOOG|         31.82|
|2019| NFLX|        139.83|
|2020| GOOG|          70.9|
|2020| NFLX|        167.45|
|2021| GOOG|         46.09|
|2021| NFLX|         99.57|
+----+-----+--------------+
```



### DataFrame Solutions

In [242]:
dfg = stocksDF.selectExpr('Date','Stock as Stock1','Open as Open1','Close as Close1',"ROUND((Close-Open)/Open*100,2) as NetChange1").filter(col('Stock') == 'GOOG')
dfg.show(5)



+----------+------+------+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|
+----------+------+------+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|
|2013-01-04|  GOOG|363.31|367.61|      1.18|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|
+----------+------+------+------+----------+
only showing top 5 rows



In [243]:
dfn = stocksDF.selectExpr('Date','Stock as Stock2','Open as Open2','Close as Close2',"ROUND((Close-Open)/Open*100,2) as NetChange2").filter(col('Stock') == 'NFLX')
dfn.show(5)


+----------+------+-----+------+----------+
|      Date|Stock2|Open2|Close2|NetChange2|
+----------+------+-----+------+----------+
|2013-01-02|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  NFLX|14.29| 13.88|     -2.87|
+----------+------+-----+------+----------+
only showing top 5 rows



In [244]:
netchanges = dfg.join(dfn, on = 'Date')
netchanges.show(5)

+----------+------+------+------+----------+------+-----+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|Stock2|Open2|Close2|NetChange2|
+----------+------+------+------+----------+------+-----+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  GOOG|363.31|367.61|      1.18|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|  NFLX|14.29| 13.88|     -2.87|
+----------+------+------+------+----------+------+-----+------+----------+
only showing top 5 rows



In [265]:
high_g = netchanges.filter(col("NetChange1") > col("NetChange2")).selectExpr('Date','Stock1 as Stock','Open1 as Open','Close1 as Close','NetChange1 as NetChange')
high_n = netchanges.filter(col("NetChange1") <= col("NetChange2")).selectExpr('Date','Stock2 as Stock','Open2 as Open','Close2 as Close','NetChange2 as NetChange')

HighNC = high_n.union(high_g).sort('Date')
HighNC.show()


+----------+-----+------+------+---------+
|      Date|Stock|  Open| Close|NetChange|
+----------+-----+------+------+---------+
|2013-01-02| GOOG|358.37|360.27|     0.53|
|2013-01-03| NFLX| 13.14|  13.8|     5.02|
|2013-01-04| GOOG|363.31|367.61|     1.18|
|2013-01-07| NFLX| 13.77| 14.17|      2.9|
|2013-01-08| GOOG| 366.4|365.28|    -0.31|
|2013-01-09| GOOG|364.77|367.68|      0.8|
|2013-01-10| NFLX|  13.8|  14.0|     1.45|
|2013-01-11| NFLX| 14.03| 14.47|     3.14|
|2013-01-14| NFLX| 14.43| 14.78|     2.43|
|2013-01-15| GOOG|358.32|361.11|     0.78|
|2013-01-16| GOOG|359.85|356.26|     -1.0|
|2013-01-17| NFLX| 14.01| 13.96|    -0.36|
|2013-01-18| GOOG|353.85|350.94|    -0.82|
|2013-01-22| GOOG|351.01|350.12|    -0.25|
|2013-01-23| NFLX| 13.88| 14.75|     6.27|
|2013-01-24| NFLX| 20.57| 20.98|     1.99|
|2013-01-25| NFLX| 20.81| 24.22|    16.39|
|2013-01-28| GOOG|374.48|373.96|    -0.14|
|2013-01-29| NFLX| 22.59| 24.16|     6.95|
|2013-01-30| GOOG|375.46|375.51|     0.01|
+----------

In [266]:
yearNC = HighNC.selectExpr('year(Date) as Year','Stock','NetChange')
yearNC.show()

+----+-----+---------+
|Year|Stock|NetChange|
+----+-----+---------+
|2013| GOOG|     0.53|
|2013| NFLX|     5.02|
|2013| GOOG|     1.18|
|2013| NFLX|      2.9|
|2013| GOOG|    -0.31|
|2013| GOOG|      0.8|
|2013| NFLX|     1.45|
|2013| NFLX|     3.14|
|2013| NFLX|     2.43|
|2013| GOOG|     0.78|
|2013| GOOG|     -1.0|
|2013| NFLX|    -0.36|
|2013| GOOG|    -0.82|
|2013| GOOG|    -0.25|
|2013| NFLX|     6.27|
|2013| NFLX|     1.99|
|2013| NFLX|    16.39|
|2013| GOOG|    -0.14|
|2013| NFLX|     6.95|
|2013| GOOG|     0.01|
+----+-----+---------+
only showing top 20 rows



In [267]:
from pyspark.sql.functions import sum, round
yearNC.groupBy('Year','Stock').agg(round(sum('NetChange'),2).alias('TotalNetChange')).sort('Year','Stock').show() 

                                                                     
                                                                     

+----+-----+--------------+
|Year|Stock|TotalNetChange|
+----+-----+--------------+
|2013| GOOG|         26.02|
|2013| NFLX|        241.05|
|2014| GOOG|         -4.03|
|2014| NFLX|        141.55|
|2015| GOOG|         30.11|
|2015| NFLX|        185.94|
|2016| GOOG|          0.72|
|2016| NFLX|         168.4|
|2017| GOOG|         17.07|
|2017| NFLX|        109.73|
|2018| GOOG|          -8.9|
|2018| NFLX|        190.29|
|2019| GOOG|         31.82|
|2019| NFLX|        139.83|
|2020| GOOG|          70.9|
|2020| NFLX|        167.45|
|2021| GOOG|         46.09|
|2021| NFLX|         99.57|
+----+-----+--------------+



### SQL Solutions

In [189]:
df_A = spark.sql("""
SELECT Date, Stock as Stock1, Open as Open1, Close as Close1, ROUND((Close-Open)/Open*100,2) as NetChange1
    FROM stocks_table
    WHERE Stock == 'GOOG'

""")

df_A.show(5)

+----------+------+------+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|
+----------+------+------+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|
|2013-01-04|  GOOG|363.31|367.61|      1.18|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|
+----------+------+------+------+----------+
only showing top 5 rows



In [190]:
df_B = spark.sql("""
SELECT Date, Stock as Stock2, Open as Open2, Close as Close2, ROUND((Close-Open)/Open*100,2) as NetChange2
    FROM stocks_table
    WHERE Stock == 'NFLX'

""")

df_B.show(5)

+----------+------+-----+------+----------+
|      Date|Stock2|Open2|Close2|NetChange2|
+----------+------+-----+------+----------+
|2013-01-02|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  NFLX|14.29| 13.88|     -2.87|
+----------+------+-----+------+----------+
only showing top 5 rows



In [198]:
df_A.createOrReplaceTempView('tableA')
df_B.createOrReplaceTempView('tableB')

df_c = spark.sql("""
    SELECT tableA.Date, Stock1,Open1,Close1,NetChange1,Stock2,Open2,Close2,NetChange2 FROM 
        tableA INNER JOIN tableB
            ON tableA.Date = tableB.Date
""")

df_c.show(5)

+----------+------+------+------+----------+------+-----+------+----------+
|      Date|Stock1| Open1|Close1|NetChange1|Stock2|Open2|Close2|NetChange2|
+----------+------+------+------+----------+------+-----+------+----------+
|2013-01-02|  GOOG|358.37|360.27|      0.53|  NFLX| 13.6| 13.14|     -3.38|
|2013-01-03|  GOOG|361.11|360.48|     -0.17|  NFLX|13.14|  13.8|      5.02|
|2013-01-04|  GOOG|363.31|367.61|      1.18|  NFLX|13.79| 13.71|     -0.58|
|2013-01-07|  GOOG|366.35| 366.0|      -0.1|  NFLX|13.77| 14.17|       2.9|
|2013-01-08|  GOOG| 366.4|365.28|     -0.31|  NFLX|14.29| 13.88|     -2.87|
+----------+------+------+------+----------+------+-----+------+----------+
only showing top 5 rows



In [268]:
df_c.createOrReplaceTempView('tableC')

df_d = spark.sql("""
SELECT Date,Stock, Open, Close, NetChange FROM
(SELECT Date,Stock1 as Stock, Open1 as Open, Close1 as Close, NetChange1 as NetChange
    FROM tableC
    WHERE NetChange1 > NetChange2)
UNION
(SELECT Date,Stock2 as Stock, Open2 as Open, Close2 as Close, NetChange2 as NetChange
    FROM tableC
    WHERE NetChange1 <= NetChange2)
        ORDER BY Date

""")

df_d.show()





+----------+-----+------+------+---------+
|      Date|Stock|  Open| Close|NetChange|
+----------+-----+------+------+---------+
|2013-01-02| GOOG|358.37|360.27|     0.53|
|2013-01-03| NFLX| 13.14|  13.8|     5.02|
|2013-01-04| GOOG|363.31|367.61|     1.18|
|2013-01-07| NFLX| 13.77| 14.17|      2.9|
|2013-01-08| GOOG| 366.4|365.28|    -0.31|
|2013-01-09| GOOG|364.77|367.68|      0.8|
|2013-01-10| NFLX|  13.8|  14.0|     1.45|
|2013-01-11| NFLX| 14.03| 14.47|     3.14|
|2013-01-14| NFLX| 14.43| 14.78|     2.43|
|2013-01-15| GOOG|358.32|361.11|     0.78|
|2013-01-16| GOOG|359.85|356.26|     -1.0|
|2013-01-17| NFLX| 14.01| 13.96|    -0.36|
|2013-01-18| GOOG|353.85|350.94|    -0.82|
|2013-01-22| GOOG|351.01|350.12|    -0.25|
|2013-01-23| NFLX| 13.88| 14.75|     6.27|
|2013-01-24| NFLX| 20.57| 20.98|     1.99|
|2013-01-25| NFLX| 20.81| 24.22|    16.39|
|2013-01-28| GOOG|374.48|373.96|    -0.14|
|2013-01-29| NFLX| 22.59| 24.16|     6.95|
|2013-01-30| GOOG|375.46|375.51|     0.01|
+----------

In [269]:
df_d.createOrReplaceTempView('tableD')

spark.sql("""
    SELECT year(Date) as Year, Stock,NetChange FROM tableD
""").show()


+----+-----+---------+
|Year|Stock|NetChange|
+----+-----+---------+
|2013| GOOG|     0.53|
|2013| NFLX|     5.02|
|2013| GOOG|     1.18|
|2013| NFLX|      2.9|
|2013| GOOG|    -0.31|
|2013| GOOG|      0.8|
|2013| NFLX|     1.45|
|2013| NFLX|     3.14|
|2013| NFLX|     2.43|
|2013| GOOG|     0.78|
|2013| GOOG|     -1.0|
|2013| NFLX|    -0.36|
|2013| GOOG|    -0.82|
|2013| GOOG|    -0.25|
|2013| NFLX|     6.27|
|2013| NFLX|     1.99|
|2013| NFLX|    16.39|
|2013| GOOG|    -0.14|
|2013| NFLX|     6.95|
|2013| GOOG|     0.01|
+----+-----+---------+
only showing top 20 rows



In [270]:
spark.sql("""
    SELECT year(Date) as Year, Stock,ROUND(sum(NetChange),2) as TotalNetChange FROM tableD
    GROUP BY Year, Stock
    ORDER BY Year
""").show()

+----+-----+--------------+
|Year|Stock|TotalNetChange|
+----+-----+--------------+
|2013| NFLX|        241.05|
|2013| GOOG|         26.02|
|2014| NFLX|        141.55|
|2014| GOOG|         -4.03|
|2015| NFLX|        185.94|
|2015| GOOG|         30.11|
|2016| NFLX|         168.4|
|2016| GOOG|          0.72|
|2017| NFLX|        109.73|
|2017| GOOG|         17.07|
|2018| NFLX|        190.29|
|2018| GOOG|          -8.9|
|2019| NFLX|        139.83|
|2019| GOOG|         31.82|
|2020| NFLX|        167.45|
|2020| GOOG|          70.9|
|2021| NFLX|         99.57|
|2021| GOOG|         46.09|
+----+-----+--------------+



In [ ]:
spark.stop()